In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import seaborn as sns
from sklearn import preprocessing

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

sub = open('sub.csv', 'w')

In [ ]:
#find out which columns the train set has
#group them according to their types

In [3]:
train.columns.groupby(train.dtypes)

{dtype('int64'): Index(['PassengerId', 'Survived', 'Pclass', 'SibSp', 'Parch'], dtype='object'),
 dtype('float64'): Index(['Age', 'Fare'], dtype='object'),
 dtype('O'): Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')}

In [ ]:
#concatenate train set and test set

In [4]:
y = pd.DataFrame(train.Survived)
n_train = train.shape[0]
passenderId = test['PassengerId']
data = pd.concat((train, test)).reset_index(drop=True)
print("data: ",data.shape, "\ntrain: ", train.shape, "\ntest: ", test.shape)

data:  (1309, 12) 
train:  (891, 12) 
test:  (418, 11)


In [ ]:
#extract columns that have null elements

In [5]:
null_data = (data.isnull().sum() / len(data)) * 100
null_data = null_data.drop(null_data[null_data == 0].index).sort_values(ascending=True)
null_data

Fare         0.076394
Embarked     0.152788
Age         20.091673
Survived    31.932773
Cabin       77.463713
dtype: float64

In [ ]:
#describe data according to their mean, top value, and count

In [6]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
train.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Nenkoff, Mr. Christo",male,1601,B96 B98,S
freq,1,577,7,4,644


In [8]:
train[['Pclass', 'Survived']].groupby(['Pclass']).mean()

,Survived
Pclass,
1,0.629630
2,0.472826
3,0.242363


In [9]:
train[['SibSp', 'Survived']].groupby('SibSp').mean()

,Survived
SibSp,
0,0.345395
1,0.535885
2,0.464286
3,0.250000
4,0.166667
5,0.000000
8,0.000000


In [10]:
train[['Parch', 'Survived']].groupby('Parch').mean()

,Survived
Parch,
0,0.343658
1,0.550847
2,0.500000
3,0.600000
4,0.000000
5,0.200000
6,0.000000


In [11]:
train[['Sex', 'Survived']].groupby('Sex').mean()

,Survived
Sex,
female,0.742038
male,0.188908


In [12]:
train[['Embarked', 'Survived']].groupby('Embarked').mean()

,Survived
Embarked,
C,0.553571
Q,0.389610
S,0.336957


In [ ]:
#handle null elements either with fill them or drop them 

In [13]:
data['Age'] = data['Age'].fillna(data.Age.mean())
data['Fare'] = data['Fare'].fillna(data.Fare.mean())
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
data.drop(['Cabin', 'PassengerId', 'Ticket', 'Name', 'Survived'], axis=1, inplace=True)

In [14]:
data.shape

(1309, 7)

In [15]:
null_data = (data.isnull().sum() / len(data)) * 100
null_data = null_data.drop(null_data[null_data == 0].index).sort_values(ascending=True)
null_data

Series([], dtype: float64)

In [ ]:
#transform columns that contains objects to the encoded form

In [16]:
lbl = preprocessing.LabelEncoder()
col = data.select_dtypes(include=['object'])
for labels in col:
    data[labels] = lbl.fit_transform(data[labels])

In [ ]:
#split test and train set 

In [17]:
X_train = data[:n_train]
X_test = data[n_train:]
print("train,", X_train.shape, "\ntest, ", X_test.shape)

train, (891, 7) 
test,  (418, 7)


In [ ]:
#use Random Forest Classifier

In [18]:
rndm = RandomForestClassifier()
rndm.fit(X_train, y)
pred = rndm.predict(X_test)

C:\Users\AFRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [19]:
from sklearn.metrics import accuracy_score

In [20]:
sol = pd.read_csv('gender_submission.csv')
sol.drop('PassengerId', axis=1, inplace=True)
accuracy_score(sol, pred)

0.83732057416267947

In [ ]:
#use Gradient Boosting Classifier

In [21]:
grb = GradientBoostingClassifier()
grb.fit(X_train, y)
predicted = grb.predict(X_test)
accuracy_score(sol, predicted)

C:\Users\AFRA\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.88755980861244022

In [22]:
sol = pd.DataFrame({'PassengerId': passenderId, 'Survived': predicted})
sol.to_csv(sub, index=False)
sub.close()